# 🎬 Netflix Churn in BigQuery ML — Colab Notebook

This notebook walks you through a **step-by-step (9-cell) pipeline** to build a **features + label** table (`feat_churn_lite`) from `users` and `watch_history`, followed by a **minimal logistic regression** using **BigQuery ML**.

We keep complexity at **Level 4**: basic joins, rolling features, and next-month churn labeling — perfect for a warm-up before adding Gemini-assisted improvements.

## Lab Objectives
Build confidence with SQL progressively (Levels 0–4): counts, grouping, date truncation, joins, and window functions.

Construct a user×month panel that includes zero-activity months and create rolling 3-month features (r3_sess, r3_min).
Define a clean next-month churn label (inactive next month = 1) without data leakage.

Orchestrate a reproducible Colab → BigQuery workflow that materializes each step as a table.

Train a minimal BigQuery ML logistic regression, evaluate it (AUC, log loss, ROC/PR, confusion matrix), and generate a scored table for the latest month.
Practice model discipline: small feature set, clear assumptions, sanity checks, and an auditable pipeline.

Use Gemini to synthesize a single-query CTE equivalent of your 9-step pipeline and verify equivalence.



## Problem Statement
You are the data analyst for a streaming platform with three core tables: users, watch_history, and (optionally later) movies. Product leadership wants an early warning signal for which users are likely to be inactive next month so lifecycle marketing can target preventive outreach. Your task is to build a transparent, baseline churn pipeline—staying:
* Creates a canonical calendar of months and a user×month grid,
* Aggregates watch behavior to monthly sessions and minutes, filling zero-activity months,
* Engineers minimal, interpretable rolling features (r3_sess, r3_min),
* Defines the label churn_next_month = 1 − active_next_month,
* Trains and evaluates a logistic regression using only r3_sess, r3_min, subscription_plan, country, and age,
* Produces ranked churn probabilities for the most recent month, and
Uses Gemini to compress the pipeline into one CTE query and verify it matches your materialized table.


## Constraints
Use only past and present information to predict the next month (no peeking into the future).
Keep features minimal and documented; label logic must be auditable.
Deliverables:
* A BigQuery table feat_churn_lite (features + label).
* A trained model churn_logreg_lite with evaluation metrics.
* A predictions table for the latest month with churn probabilities.
* A Gemini-generated single-query CTE and an equivalence check showing parity with your pipeline.

Baseline target: model performance meaningfully above random (e.g., AUC > 0.60) with clear notes on next steps to improve.

## ⬜️ Cell 0 — Setup

- Authenticate to Google Cloud from Colab  
- Initialize BigQuery client  
- Set your **Project ID** and **Dataset** (edit these two variables)

## Install libraries

In [1]:
!pip -q install google-cloud-bigquery pandas --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 44.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.


## Authenticate

In [2]:
# --- Authenticate to Google Cloud from Colab (you'll get a popup) ---
from google.colab import auth
auth.authenticate_user()

# --- Basic imports ---
import os
from google.cloud import bigquery

# --- EDIT THESE THREE ONLY ---
PROJECT_ID = "mgmt-467-nh"   # your GCP project id
DATASET    = "netflix"          # your BigQuery dataset
REGION     = "us-central1"      # the dataset's location (check in BigQuery UI)

# Save for later string formatting if needed
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["DATASET"]    = DATASET
os.environ["REGION"]     = REGION
print(PROJECT_ID, DATASET, REGION)

mgmt-467-nh netflix us-central1


## ⬜️ Cell 1 — Month Bounds

Find the **minimum** and **maximum** months in `watch_history`.  
We’ll use these to generate a canonical calendar of months that covers the entire activity range.

In [3]:
import pandas as pd
bq = bigquery.Client(project=PROJECT_ID)

In [4]:
sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.month_bounds` AS
SELECT
  DATE_TRUNC(MIN(DATE(watch_date)), MONTH) AS min_month,
  DATE_TRUNC(MAX(DATE(watch_date)), MONTH) AS max_month
FROM `{PROJECT_ID}.{DATASET}.watch_history`;
"""
bq.query(sql).result()
print("✓ month_bounds")

✓ month_bounds


##Check if the table is creatred

In [5]:
sql = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET}.month_bounds`
"""

bq.query(sql).result().to_dataframe()

,min_month,max_month
0,2024-01-01,2025-12-01


## ⬜️ Cell 2 — Calendar of Months

Create a dense **month calendar** from `min_month` to `max_month`.  
This ensures we consider **months with zero activity**, which are crucial for churn labeling.

In [6]:
sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.calendar_months` AS
WITH bounds AS (
  SELECT min_month, max_month FROM `{PROJECT_ID}.{DATASET}.month_bounds`
),
seq AS (
  SELECT GENERATE_DATE_ARRAY(
    (SELECT min_month FROM bounds),
    (SELECT max_month FROM bounds),
    INTERVAL 1 MONTH
  ) AS months
)
SELECT month
FROM seq, UNNEST(months) AS month;
"""
bq.query(sql).result()
print("✓ calendar_months")

✓ calendar_months


## Check if the table was created

In [7]:
sql = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET}.calendar_months`
ORDER BY month
Limit 5
"""

bq.query(sql).result().to_dataframe()

,month
0,2024-01-01
1,2024-02-01
2,2024-03-01
3,2024-04-01
4,2024-05-01


## ⬜️ Cell 3 — User × Month Grid

Build a canonical **user × month** grid so every user has a row for every month, even if they had **no sessions**.

In [8]:
sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.user_month_grid` AS
SELECT u.user_id, c.month
FROM (SELECT DISTINCT user_id FROM `{PROJECT_ID}.{DATASET}.users`) u
CROSS JOIN `{PROJECT_ID}.{DATASET}.calendar_months` c;
"""
bq.query(sql).result()
print("✓ user_month_grid")

✓ user_month_grid


In [9]:
sql = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET}.user_month_grid`
LIMIT 5;
"""
bq.query(sql).result().to_dataframe()

,user_id,month
0,user_03959,2024-01-01
1,user_03263,2024-01-01
2,user_04177,2024-01-01
3,user_00822,2024-01-01
4,user_00451,2024-01-01


## ⬜️ Cell 4 — Monthly Activity Aggregation

Aggregate `watch_history` to **monthly sessions** and **minutes** per user.  
We do not use rolling sums yet; just raw monthly totals.

In [10]:
sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.activity_monthly` AS
SELECT
  user_id,
  DATE_TRUNC(DATE(watch_date), MONTH) AS month,
  COUNT(*) AS sessions,
  SUM(watch_duration_minutes) AS minutes
FROM `{PROJECT_ID}.{DATASET}.watch_history`
GROUP BY 1,2;
"""
bq.query(sql).result()
print("✓ activity_monthly")

✓ activity_monthly


## Display 5 records

In [11]:
sql = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET}.activity_monthly`
LIMIT 5;
"""
bq.query(sql).result().to_dataframe()

,user_id,month,sessions,minutes
0,user_02817,2024-01-01,2,298.8
1,user_02658,2024-01-01,2,20.0
2,user_00144,2024-01-01,2,150.0
3,user_09248,2024-01-01,2,NaN
4,user_03505,2024-01-01,2,64.0


## ⬜️ Cell 5 — Fill Zero-Activity Months

Left-join the monthly activity onto the **user × month grid** and fill missing as zeros.  
This step ensures **churn** (no activity next month) can be correctly labeled.

In [12]:
sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.activity_filled` AS
SELECT
  g.user_id,
  g.month,
  COALESCE(a.sessions, 0) AS sessions,
  COALESCE(a.minutes, 0)  AS minutes
FROM `{PROJECT_ID}.{DATASET}.user_month_grid` g
LEFT JOIN `{PROJECT_ID}.{DATASET}.activity_monthly` a
USING (user_id, month);
"""
bq.query(sql).result()
print("✓ activity_filled")

✓ activity_filled


In [13]:
sql = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET}.activity_filled`
WHERE sessions = 0 AND minutes = 0
LIMIT 5;
"""
bq.query(sql).result().to_dataframe()

,user_id,month,sessions,minutes
0,user_05946,2024-01-01,0,0.0
1,user_07336,2024-01-01,0,0.0
2,user_09945,2024-01-01,0,0.0
3,user_00013,2024-01-01,0,0.0
4,user_09712,2024-01-01,0,0.0


## ⬜️ Cell 6 — Rolling 3-Month Features

**Feature Engineering:** Compute **recency/frequency** (`r3_sess`) and **intensity** (`r3_min`) as rolling 3-month sums per user.  
These are compact but strong baseline features for churn prediction.

In [14]:
sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.activity_roll3` AS
SELECT
  user_id,
  month,
  sessions,
  minutes,
  SUM(sessions) OVER (
    PARTITION BY user_id ORDER BY month
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
  ) AS r3_sess,
  SUM(minutes) OVER (
    PARTITION BY user_id ORDER BY month
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
  ) AS r3_min
FROM `{PROJECT_ID}.{DATASET}.activity_filled`;
"""
bq.query(sql).result()
print("✓ activity_roll3")

✓ activity_roll3


In [15]:
sql = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET}.activity_roll3`
LIMIT 5;
"""
bq.query(sql).result().to_dataframe()

,user_id,month,sessions,minutes,r3_sess,r3_min
0,user_04724,2024-01-01,0,0.0,0,0.0
1,user_06189,2024-01-01,0,0.0,0,0.0
2,user_05720,2024-01-01,0,0.0,0,0.0
3,user_03168,2024-01-01,0,0.0,0,0.0
4,user_06469,2024-01-01,0,0.0,0,0.0


## ⬜️ Cell 7 — Label: Next-Month Churn

Define **active next month** = 1 if a user has **any sessions** next month, else 0.  
Our label **`churn_next_month` = 1 - active_next_month** (1 means churn).

In [16]:
sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.labels_next_month` AS
SELECT
  user_id,
  month,
  LEAD(CASE WHEN sessions > 0 THEN 1 ELSE 0 END)
    OVER (PARTITION BY user_id ORDER BY month) AS active_next_month
FROM `{PROJECT_ID}.{DATASET}.activity_filled`;
"""
bq.query(sql).result()
print("✓ labels_next_month")

✓ labels_next_month


In [17]:
sql = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET}.labels_next_month`
LIMIT 5;
"""
bq.query(sql).result().to_dataframe()

,user_id,month,active_next_month
0,user_00920,2025-12-01,<NA>
1,user_03933,2025-12-01,<NA>
2,user_05799,2025-12-01,<NA>
3,user_02563,2025-12-01,<NA>
4,user_05265,2025-12-01,<NA>


## ⬜️ Cell 8 — User Attributes (Lite)

Bring in simple demographic/product context: `subscription_plan`, `country`, `age`.  
These keep the feature set small and interpretable.

In [18]:
sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.user_attrs_lite` AS
SELECT user_id, subscription_plan, country, age
FROM `{PROJECT_ID}.{DATASET}.users`;
"""
bq.query(sql).result()
print("✓ user_attrs_lite")

✓ user_attrs_lite


In [19]:
sql = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET}.user_attrs_lite`
LIMIT 5;
"""
bq.query(sql).result().to_dataframe()

,user_id,subscription_plan,country,age
0,user_00008,Basic,Canada,NaN
1,user_00008,Basic,Canada,NaN
2,user_00024,Standard,Canada,NaN
3,user_00024,Standard,Canada,NaN
4,user_00028,Standard,USA,NaN


## ⬜️ Cell 9 — Assemble Final Feature + Label Table

Join rolling features + user attributes + labels to build `feat_churn_lite`.  
We drop rows where the label is `NULL` (i.e., there is no “next month” to label).

In [20]:
sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.feat_churn_lite` AS
SELECT
  r.user_id,
  r.month,
  r.r3_sess,
  r.r3_min,
  u.subscription_plan,
  u.country,
  u.age,
  CASE
    WHEN l.active_next_month = 1 THEN 0
    WHEN l.active_next_month = 0 THEN 1
    ELSE NULL
  END AS churn_next_month
FROM `{PROJECT_ID}.{DATASET}.activity_roll3` r
JOIN `{PROJECT_ID}.{DATASET}.user_attrs_lite` u USING (user_id)
JOIN `{PROJECT_ID}.{DATASET}.labels_next_month` l USING (user_id, month)
WHERE l.active_next_month IS NOT NULL;
"""
bq.query(sql).result()
print("✓ feat_churn_lite")

✓ feat_churn_lite


In [21]:
sql = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite`
LIMIT 5;
"""
bq.query(sql).result().to_dataframe()

,user_id,month,r3_sess,r3_min,subscription_plan,country,age,churn_next_month
0,user_00001,2024-10-01,0,0.0,Basic,USA,43.0,0
1,user_00001,2024-01-01,0,0.0,Basic,USA,43.0,0
2,user_00001,2024-01-01,0,0.0,Basic,USA,43.0,0
3,user_00001,2024-10-01,0,0.0,Basic,USA,43.0,0
4,user_00002,2024-08-01,0,0.0,Premium+,USA,38.0,1


## ✅ Quick Sanity Checks (Optional)

Confirm the label is present, inspect class balance, and ensure features are populated.

In [22]:
checks = f"""
SELECT 'rows_total' AS check_name, COUNT(*) AS val FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite`
UNION ALL
SELECT 'null_labels', COUNTIF(churn_next_month IS NULL) FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite`
UNION ALL
SELECT 'class_0', COUNT(*) FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite` WHERE churn_next_month=0
UNION ALL
SELECT 'class_1', COUNT(*) FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite` WHERE churn_next_month=1;
"""
from pandas import set_option
set_option('display.max_rows', 10)
bq.query(checks).result().to_dataframe()

,check_name,val
0,rows_total,473800
1,null_labels,0
2,class_0,161284
3,class_1,312516


# 🔮 Logistic Regression (BigQuery ML)

We now train a **minimal logistic regression** using the features we built, evaluate it, and generate predictions for the last month.

## ⬜️ Train — Minimal Model (AUTO Split)

Uses: `r3_sess`, `r3_min`, `subscription_plan`, `country`, `age`  
Label: `churn_next_month`

In [23]:
sql = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET}.churn_logreg_lite`
OPTIONS (
  model_type = 'logistic_reg',
  input_label_cols = ['churn_next_month'],
  data_split_method = 'SEQ',
  data_split_col = 'month' -- Use the 'month' column for sequential splitting
) AS
SELECT
  churn_next_month,
  r3_sess, r3_min,
  subscription_plan, country, age,
  month -- Include month in the select list for SEQ split
FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite`
ORDER BY month; -- Order by month for sequential split
"""
bq.query(sql).result()
print("✓ trained model:", f"{PROJECT_ID}.{DATASET}.churn_logreg_lite")

✓ trained model: mgmt-467-nh.netflix.churn_logreg_lite


## ⬜️ Evaluate — Metrics & Curves

We’ll look at AUC and log_loss, plus fetch ROC/PR curve points.  
Students can plot these in later exercises if desired.

In [24]:
eval_df = bq.query(
    f"SELECT * FROM ML.EVALUATE(MODEL `{PROJECT_ID}.{DATASET}.churn_logreg_lite`);"
).result().to_dataframe()
eval_df

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.66068,1.0,0.66068,0.795674,0.640611,0.501681


In [25]:
roc_df = bq.query(
    f"SELECT * FROM ML.ROC_CURVE(MODEL `{PROJECT_ID}.{DATASET}.churn_logreg_lite`);"
).result().to_dataframe()
roc_df.head()

,threshold,recall,false_positive_rate,true_positives,false_positives,true_negatives,false_negatives
0,0.661355,0.910301,0.909471,61946,31786,3164,6104
1,0.663214,0.380338,0.379628,25882,13268,21682,42168
2,0.662724,0.481470,0.478169,32764,16712,18238,35286
3,0.665938,0.180279,0.179914,12268,6288,28662,55782
4,0.662677,0.491315,0.487954,33434,17054,17896,34616


In [26]:
f"""
SELECT * FROM ML.PR_CURVE(MODEL `{PROJECT_ID}.{DATASET}.churn_logreg_lite`)
"""
pr_df = bq.query(sql).result().to_dataframe()
pr_df.head()

""


In [27]:
cm_df = bq.query(f"""
SELECT *
FROM ML.CONFUSION_MATRIX(
  MODEL `{PROJECT_ID}.{DATASET}.churn_logreg_lite`,
  (SELECT
     churn_next_month,
     r3_sess, r3_min, subscription_plan, country, age
   FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite`),
  STRUCT(0.5 AS threshold)
);
""").result().to_dataframe()
cm_df

,expected_label,_0,_1
0,0,0,161284
1,1,0,312516


## ⬜️ Predict — Score the Latest Month

We score the most recent month in `feat_churn_lite` and save predictions to a table for inspection or dashboards.

In [28]:
# Helper: last month to score
bq.query(f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.score_month` AS
SELECT MAX(month) AS score_month
FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite`;
""").result()

# Predictions table
bq.query(f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.churn_predictions_lite` AS
WITH to_score AS (
  SELECT f.*
  FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite` f
  CROSS JOIN `{PROJECT_ID}.{DATASET}.score_month` s
  WHERE f.month = s.score_month
)
SELECT
  user_id,
  month AS score_month,
  predicted_churn_next_month AS yhat,
  (SELECT prob FROM UNNEST(predicted_churn_next_month_probs) WHERE label = predicted_churn_next_month) AS prob_churn
FROM ML.PREDICT(
  MODEL `{PROJECT_ID}.{DATASET}.churn_logreg_lite`,
  TABLE to_score
);
""").result()

bq.query(
    f"SELECT * FROM `{PROJECT_ID}.{DATASET}.churn_predictions_lite` ORDER BY prob_churn DESC LIMIT 20"
).result().to_dataframe()

,user_id,score_month,yhat,prob_churn
0,user_09287,2025-11-01,1,0.673125
1,user_09287,2025-11-01,1,0.673125
2,user_04039,2025-11-01,1,0.670733
3,user_04039,2025-11-01,1,0.670733
4,user_07026,2025-11-01,1,0.670501
...,...,...,...,...
15,user_00331,2025-11-01,1,0.670059
16,user_06101,2025-11-01,1,0.670015
17,user_06101,2025-11-01,1,0.670015
18,user_01461,2025-11-01,1,0.669857


## Next Steps
* Analyze current features
* Brainstorm new features
* Feature engineering
* Iterate on the model
* Evaluate the improved model
* Refine features and model
* Finish task

---

## 🤖 Gemini Synthesis & Compare Prompt

Use this prompt with Gemini (in the side panel or a text cell) to generate a **single-query CTE** version of your 9-step pipeline and then **compare results** against your materialized table.

**Copy/paste this prompt:**

```
You are a data engineering copilot. I built a 9-step BigQuery pipeline that creates `${PROJECT_ID}.${DATASET}.feat_churn_lite` with features (r3_sess, r3_min, subscription_plan, country, age) and label churn_next_month, using:
1) month_bounds
2) calendar_months
3) user_month_grid
4) activity_monthly
5) activity_filled
6) activity_roll3
7) labels_next_month
8) user_attrs_lite
9) final select

Task A: Produce a single BigQuery SQL (CTEs only; no CREATE TABLE) that computes the same final columns directly from `${PROJECT_ID}.${DATASET}.users` and `${PROJECT_ID}.${DATASET}.watch_history` with identical logic: calendar, user×month grid, zero fill, rolling 3-month sums, and label from next-month activity.

Task B: Produce a second SQL snippet that checks equivalence vs the materialized table:
- Row count difference
- Class distribution difference
- A checksum hash over ordered (user_id, month)

Return only two code blocks titled:
-- SINGLE-QUERY CTE VERSION
-- EQUIVALENCE CHECKS
Use fully qualified names with my PROJECT_ID and DATASET placeholders.
```

In [29]:
sql = f"""
-- SINGLE-QUERY CTE VERSION
WITH month_bounds AS (
  SELECT
    DATE_TRUNC(MIN(DATE(watch_date)), MONTH) AS min_month,
    DATE_TRUNC(MAX(DATE(watch_date)), MONTH) AS max_month
  FROM `{PROJECT_ID}.{DATASET}.watch_history`
),
calendar_months AS (
  SELECT month
  FROM month_bounds,
       UNNEST(GENERATE_DATE_ARRAY(min_month, max_month, INTERVAL 1 MONTH)) AS month
),
user_month_grid AS (
  SELECT u.user_id, c.month
  FROM (SELECT DISTINCT user_id FROM `{PROJECT_ID}.{DATASET}.users`) u
  CROSS JOIN calendar_months c
),
activity_monthly AS (
  SELECT
    user_id,
    DATE_TRUNC(DATE(watch_date), MONTH) AS month,
    COUNT(*) AS sessions,
    SUM(watch_duration_minutes) AS minutes
  FROM `{PROJECT_ID}.{DATASET}.watch_history`
  GROUP BY 1, 2
),
activity_filled AS (
  SELECT
    g.user_id,
    g.month,
    COALESCE(a.sessions, 0) AS sessions,
    COALESCE(a.minutes, 0)  AS minutes
  FROM user_month_grid g
  LEFT JOIN activity_monthly a USING (user_id, month)
),
activity_roll3 AS (
  SELECT
    user_id,
    month,
    sessions,
    minutes,
    SUM(sessions) OVER (
      PARTITION BY user_id ORDER BY month
      ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS r3_sess,
    SUM(minutes) OVER (
      PARTITION BY user_id ORDER BY month
      ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS r3_min
  FROM activity_filled
),
labels_next_month AS (
  SELECT
    user_id,
    month,
    LEAD(CASE WHEN sessions > 0 THEN 1 ELSE 0 END)
      OVER (PARTITION BY user_id ORDER BY month) AS active_next_month
    FROM activity_filled
),
user_attrs_lite AS (
  SELECT user_id, subscription_plan, country, age
  FROM `{PROJECT_ID}.{DATASET}.users`
)
SELECT
  r.user_id,
  r.month,
  r.r3_sess,
  r.r3_min,
  u.subscription_plan,
  u.country,
  u.age,
  CASE
    WHEN l.active_next_month = 1 THEN 0
    WHEN l.active_next_month = 0 THEN 1
    ELSE NULL
  END AS churn_next_month
FROM activity_roll3 r
JOIN user_attrs_lite u USING (user_id)
JOIN labels_next_month l USING (user_id, month)
WHERE l.active_next_month IS NOT NULL;
"""
bq.query(sql).result().to_dataframe() # Execute and display results

,user_id,month,r3_sess,r3_min,subscription_plan,country,age,churn_next_month
0,user_00053,2024-01-01,0,0.0,Standard,USA,34.0,1
1,user_00053,2024-01-01,0,0.0,Standard,USA,34.0,1
2,user_00053,2024-02-01,0,0.0,Standard,USA,34.0,0
3,user_00053,2024-02-01,0,0.0,Standard,USA,34.0,0
4,user_00053,2024-03-01,4,30.4,Standard,USA,34.0,1
...,...,...,...,...,...,...,...,...
473795,user_09791,2025-09-01,0,0.0,Premium,USA,35.0,1
473796,user_09791,2025-10-01,0,0.0,Premium,USA,35.0,1
473797,user_09791,2025-10-01,0,0.0,Premium,USA,35.0,1
473798,user_09791,2025-11-01,0,0.0,Premium,USA,35.0,0


In [30]:
sql = f"""
-- EQUIVALENCE CHECKS
WITH cte_version AS (
  -- Paste the SINGLE-QUERY CTE VERSION here
  WITH month_bounds AS (
    SELECT
      DATE_TRUNC(MIN(DATE(watch_date)), MONTH) AS min_month,
      DATE_TRUNC(MAX(DATE(watch_date)), MONTH) AS max_month
    FROM `{PROJECT_ID}.{DATASET}.watch_history`
  ),
  calendar_months AS (
    SELECT month
    FROM month_bounds,
         UNNEST(GENERATE_DATE_ARRAY(min_month, max_month, INTERVAL 1 MONTH)) AS month
  ),
  user_month_grid AS (
    SELECT u.user_id, c.month
    FROM (SELECT DISTINCT user_id FROM `{PROJECT_ID}.{DATASET}.users`) u
    CROSS JOIN calendar_months c
  ),
  activity_monthly AS (
    SELECT
      user_id,
      DATE_TRUNC(DATE(watch_date), MONTH) AS month,
      COUNT(*) AS sessions,
      SUM(watch_duration_minutes) AS minutes
    FROM `{PROJECT_ID}.{DATASET}.watch_history`
    GROUP BY 1, 2
  ),
  activity_filled AS (
    SELECT
      g.user_id,
      g.month,
      COALESCE(a.sessions, 0) AS sessions,
      COALESCE(a.minutes, 0)  AS minutes
    FROM user_month_grid g
    LEFT JOIN activity_monthly a USING (user_id, month)
  ),
  activity_roll3 AS (
    SELECT
      user_id,
      month,
      sessions,
      minutes,
      SUM(sessions) OVER (
        PARTITION BY user_id ORDER BY month
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
      ) AS r3_sess,
      SUM(minutes) OVER (
        PARTITION BY user_id ORDER BY month
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
      ) AS r3_min
    FROM activity_filled
  ),
  labels_next_month AS (
    SELECT
      user_id,
      month,
      LEAD(CASE WHEN sessions > 0 THEN 1 ELSE 0 END)
        OVER (PARTITION BY user_id ORDER BY month) AS active_next_month
    FROM activity_filled
  ),
  user_attrs_lite AS (
    SELECT user_id, subscription_plan, country, age
    FROM `{PROJECT_ID}.{DATASET}.users`
  )
  SELECT
    r.user_id,
    r.month,
    r.r3_sess,
    r.r3_min,
    u.subscription_plan,
    u.country,
    u.age,
    CASE
      WHEN l.active_next_month = 1 THEN 0
      WHEN l.active_next_month = 0 THEN 1
      ELSE NULL
    END AS churn_next_month
  FROM activity_roll3 r
  JOIN user_attrs_lite u USING (user_id)
  JOIN labels_next_month l USING (user_id, month)
  WHERE l.active_next_month IS NOT NULL
),
materialized_table AS (
  SELECT user_id, month, r3_sess, r3_min, subscription_plan, country, age, churn_next_month
  FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite`
)
-- Equivalence Checks
SELECT
  'Row Count Difference' as check_name,
  (SELECT COUNT(*) FROM cte_version) - (SELECT COUNT(*) FROM materialized_table) as value
UNION ALL
SELECT
  'Class 0 Difference',
  (SELECT COUNT(*) FROM cte_version WHERE churn_next_month = 0) - (SELECT COUNT(*) FROM materialized_table WHERE churn_next_month = 0)
UNION ALL
SELECT
  'Class 1 Difference',
  (SELECT COUNT(*) FROM cte_version WHERE churn_next_month = 1) - (SELECT COUNT(*) FROM materialized_table WHERE churn_next_month = 1)
UNION ALL
SELECT
  'Checksum Hash Match (0=Match)',
  IF(
    (SELECT FARM_FINGERPRINT(STRING_AGG(TO_JSON_STRING(t) ORDER BY user_id, month)) FROM cte_version t) =
    (SELECT FARM_FINGERPRINT(STRING_AGG(TO_JSON_STRING(t) ORDER BY user_id, month)) FROM materialized_table t),
    0,
    1
  )
"""
bq.query(sql).result().to_dataframe() # Execute and display results

,check_name,value
0,Class 0 Difference,0
1,Class 1 Difference,0
2,Row Count Difference,0
3,Checksum Hash Match (0=Match),0


## Next Steps
* **Analyze current features:** Examine the existing features (r3_sess, r3_min, subscription_plan, country, age) to understand their contribution to the model and identify potential limitations.
* **Brainstorm new features:** Based on the domain knowledge of streaming services and churn behavior, brainstorm new features that could capture user engagement, content preferences, or other relevant factors. Consider features derived from the watch_history and users tables.
* **Feature engineering:** Create new features based on the brainstorming session. This might involve aggregating data, creating ratios, or extracting information from timestamps or other fields.
* **Iterate on the model:** Add the new features to the feat_churn_lite table and retrain the BigQuery ML model.
* **Evaluate the new model:** Evaluate the new model's performance using the same metrics as before (AUC, log loss, etc.) and compare it to the baseline model.

# Task
Analyze the current features in the `feat_churn_lite` table, brainstorm new features from the `watch_history` and `users` tables to improve the churn prediction model, engineer these new features, and outline a plan to iterate on the model with the new features and evaluate its performance.

## Analyze current features

### Subtask:
Examine the existing features (`r3_sess`, `r3_min`, `subscription_plan`, `country`, `age`) to understand their contribution to the model and identify potential limitations.


**Reasoning**:
Review the model evaluation metrics and confusion matrix to understand the current model's performance and the contribution of existing features.



In [31]:
cm_df = bq.query(f"""
SELECT *
FROM ML.CONFUSION_MATRIX(
  MODEL `{PROJECT_ID}.{DATASET}.churn_logreg_lite`,
  (SELECT
     churn_next_month,
     r3_sess, r3_min, subscription_plan, country, age
   FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite`),
  STRUCT(0.5 AS threshold)
);
""").result().to_dataframe()

print("Model Evaluation Metrics:")
display(eval_df)

print("\nConfusion Matrix (Threshold = 0.5):")
display(cm_df)

Model Evaluation Metrics:


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.66068,1.0,0.66068,0.795674,0.640611,0.501681



Confusion Matrix (Threshold = 0.5):


,expected_label,_0,_1
0,0,0,161284
1,1,0,312516


## Brainstorm new features

### Subtask:
Based on the domain knowledge of streaming services and churn behavior, brainstorm new features that could capture user engagement, content preferences, or other relevant factors. Consider features derived from the the `watch_history` and `users` tables.


**Reasoning**:
Brainstorm new features based on domain knowledge and available data.



In [32]:
# Brainstorming new features from watch_history and users tables

print("Potential New Features for Churn Prediction:")
print("\nEngagement-Related Features (from watch_history):")
print("- Monthly average watch duration: Captures typical viewing time per month. (Derived from watch_duration_minutes)")
print("- Number of unique days watched per month: Indicates regularity of engagement. (Derived from watch_date)")
print("- Longest continuous watching streak (in days) per month: Measures intense periods of engagement. (Derived from watch_date)")
print("- Proportion of content watched to completion (if content duration is available): Indicates content satisfaction. (Requires content duration data, not currently available)")
print("- Time since last watch: Recency of activity is a strong churn indicator. (Derived from watch_date)")
print("- Frequency of watching (e.g., average days between watches): Another measure of engagement regularity. (Derived from watch_date)")
print("- Number of different genres/categories watched per month: Measures content exploration/diversity. (Requires content genre/category data, not currently available)")

print("\nContent-Related Features (from watch_history, potentially users for preferences):")
print("- Preferred genre/category (based on watch history): Users might churn if new content in their preferred area is lacking. (Requires content genre/category data)")
print("- Proportion of watching time spent on new releases: Indicates engagement with fresh content. (Requires content release date data)")
print("- Engagement with specific popular titles/franchises: High engagement might indicate loyalty. (Requires content title/franchise data)")

print("\nUser-Related Features (from users, potentially combined with watch_history):")
print("- Tenure (months since joining): Longer tenure might indicate loyalty, but also potential for stagnation. (Derived from user join date, not currently available)")
print("- Number of devices used: Might indicate shared accounts or diverse viewing habits. (Requires device information, not currently available)")
print("- Customer support interactions (frequency, type): High frequency or certain types of interactions might precede churn. (Requires customer support data, not currently available)")
print("- Payment method/billing issues: Payment problems can lead to involuntary churn. (Requires billing data, not currently available)")

Potential New Features for Churn Prediction:

Engagement-Related Features (from watch_history):
- Monthly average watch duration: Captures typical viewing time per month. (Derived from watch_duration_minutes)
- Number of unique days watched per month: Indicates regularity of engagement. (Derived from watch_date)
- Longest continuous watching streak (in days) per month: Measures intense periods of engagement. (Derived from watch_date)
- Proportion of content watched to completion (if content duration is available): Indicates content satisfaction. (Requires content duration data, not currently available)
- Time since last watch: Recency of activity is a strong churn indicator. (Derived from watch_date)
- Frequency of watching (e.g., average days between watches): Another measure of engagement regularity. (Derived from watch_date)
- Number of different genres/categories watched per month: Measures content exploration/diversity. (Requires content genre/category data, not currently availabl

## Feature engineering

### Subtask:
Create new features based on the brainstorming session. This might involve aggregating data, creating ratios, or extracting information from timestamps or other fields.


**Reasoning**:
Modify the BigQuery SQL query to include new features derived from watch_history and users tables, specifically monthly average watch duration, number of unique days watched per month, and time since last watch. Then, execute the query to create or replace the feat_churn_lite table and verify the new columns.



In [33]:
sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.feat_churn_lite` AS
WITH month_bounds AS (
  SELECT
    DATE_TRUNC(MIN(DATE(watch_date)), MONTH) AS min_month,
    DATE_TRUNC(MAX(DATE(watch_date)), MONTH) AS max_month
  FROM `{PROJECT_ID}.{DATASET}.watch_history`
),
calendar_months AS (
  SELECT month
  FROM month_bounds,
       UNNEST(GENERATE_DATE_ARRAY(min_month, max_month, INTERVAL 1 MONTH)) AS month
),
user_month_grid AS (
  SELECT u.user_id, c.month
  FROM (SELECT DISTINCT user_id FROM `{PROJECT_ID}.{DATASET}.users`) u
  CROSS JOIN calendar_months c
),
activity_monthly AS (
  SELECT
    user_id,
    DATE_TRUNC(DATE(watch_date), MONTH) AS month,
    COUNT(*) AS sessions,
    SUM(watch_duration_minutes) AS minutes,
    COUNT(DISTINCT DATE(watch_date)) AS unique_days_watched, -- New feature: unique days watched
    AVG(watch_duration_minutes) AS avg_watch_duration -- New feature: average watch duration
  FROM `{PROJECT_ID}.{DATASET}.watch_history`
  GROUP BY 1, 2
),
activity_filled AS (
  SELECT
    g.user_id,
    g.month,
    COALESCE(a.sessions, 0) AS sessions,
    COALESCE(a.minutes, 0)  AS minutes,
    COALESCE(a.unique_days_watched, 0) AS unique_days_watched, -- New feature
    COALESCE(a.avg_watch_duration, 0) AS avg_watch_duration -- New feature
  FROM user_month_grid g
  LEFT JOIN activity_monthly a USING (user_id, month)
),
activity_roll3 AS (
  SELECT
    user_id,
    month,
    sessions,
    minutes,
    unique_days_watched, -- Include new feature
    avg_watch_duration, -- Include new feature
    -- Calculate days since last watch based on the current month and the previous month
    DATE_DIFF(month, LAG(month, 1, '1900-01-01') OVER (PARTITION BY user_id ORDER BY month), DAY) AS days_since_last_month_start, -- Days since the start of the previous month
    SUM(sessions) OVER (
      PARTITION BY user_id ORDER BY month
      ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS r3_sess,
    SUM(minutes) OVER (
      PARTITION BY user_id ORDER BY month
      ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS r3_min
  FROM activity_filled
),
labels_next_month AS (
  SELECT
    user_id,
    month,
    LEAD(CASE WHEN sessions > 0 THEN 1 ELSE 0 END)
      OVER (PARTITION BY user_id ORDER BY month) AS active_next_month
    FROM activity_filled
),
user_attrs_lite AS (
  SELECT user_id, subscription_plan, country, age
  FROM `{PROJECT_ID}.{DATASET}.users`
)
SELECT
  r.user_id,
  r.month,
  r.r3_sess,
  r.r3_min,
  r.unique_days_watched, -- Include new feature
  r.avg_watch_duration, -- Include new feature
  r.days_since_last_month_start, -- Include new feature
  u.subscription_plan,
  u.country,
  u.age,
  CASE
    WHEN l.active_next_month = 1 THEN 0
    WHEN l.active_next_month = 0 THEN 1
    ELSE NULL
  END AS churn_next_month
FROM activity_roll3 r
JOIN user_attrs_lite u USING (user_id)
JOIN labels_next_month l USING (user_id, month)
WHERE l.active_next_month IS NOT NULL;
"""
bq.query(sql).result()
print("✓ feat_churn_lite with new features")

sql = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite`
LIMIT 5;
"""
bq.query(sql).result().to_dataframe()

✓ feat_churn_lite with new features


,user_id,month,r3_sess,r3_min,unique_days_watched,avg_watch_duration,days_since_last_month_start,subscription_plan,country,age,churn_next_month
0,user_00001,2024-09-01,2,157.8,0,0.0,31,Basic,USA,43.0,1
1,user_00001,2024-04-01,2,0.0,0,0.0,31,Basic,USA,43.0,0
2,user_00001,2025-06-01,2,263.6,0,0.0,31,Basic,USA,43.0,0
3,user_00001,2024-04-01,2,0.0,0,0.0,31,Basic,USA,43.0,0
4,user_00001,2024-03-01,2,0.0,0,0.0,29,Basic,USA,43.0,1


## Iterate on the model

### Subtask:
Retrain the BigQuery ML logistic regression model with the updated `feat_churn_lite` table containing the new features.


**Reasoning**:
Construct and execute the SQL query to retrain the BigQuery ML logistic regression model with the updated feat_churn_lite table, including the new features, and print a confirmation message.



In [34]:
sql = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET}.churn_logreg_lite`
OPTIONS (
  model_type = 'logistic_reg',
  input_label_cols = ['churn_next_month'],
  data_split_method = 'SEQ',
  data_split_col = 'month'
) AS
SELECT
  churn_next_month,
  r3_sess,
  r3_min,
  unique_days_watched,  -- Include new feature
  avg_watch_duration,   -- Include new feature
  days_since_last_month_start, -- Include new feature
  subscription_plan,
  country,
  age,
  month -- Include month for SEQ split
FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite`
ORDER BY month; -- Order by month for sequential split
"""
bq.query(sql).result()
print("✓ trained model:", f"{PROJECT_ID}.{DATASET}.churn_logreg_lite with new features")

✓ trained model: mgmt-467-nh.netflix.churn_logreg_lite with new features


## Evaluate the improved model

### Subtask:
Evaluate the performance of the retrained BigQuery ML model with the new features and compare it to the baseline model.


**Reasoning**:
Query the evaluation metrics, ROC curve data, PR curve data, and confusion matrix for the retrained model using ML.EVALUATE, ML.ROC_CURVE, ML.PR_CURVE, and ML.CONFUSION_MATRIX respectively, and then display the evaluation metrics and confusion matrix dataframes to assess the performance.



In [35]:
eval_df_retrained = bq.query(
    f"SELECT * FROM ML.EVALUATE(MODEL `{PROJECT_ID}.{DATASET}.churn_logreg_lite`);"
).result().to_dataframe()

roc_df_retrained = bq.query(
    f"SELECT * FROM ML.ROC_CURVE(MODEL `{PROJECT_ID}.{DATASET}.churn_logreg_lite`);"
).result().to_dataframe()

cm_df_retrained = bq.query(f"""
SELECT *
FROM ML.CONFUSION_MATRIX(
  MODEL `{PROJECT_ID}.{DATASET}.churn_logreg_lite`,
  (SELECT
     churn_next_month,
     r3_sess, r3_min, unique_days_watched, avg_watch_duration, days_since_last_month_start,
     subscription_plan, country, age
   FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite`),
  STRUCT(0.5 AS threshold)
);
""").result().to_dataframe()

print("Retrained Model Evaluation Metrics:")
display(eval_df_retrained)

print("\nRetrained Model Confusion Matrix (Threshold = 0.5):")
display(cm_df_retrained)

print("\nBaseline Model Evaluation Metrics:")
display(eval_df)

print("\nBaseline Model Confusion Matrix (Threshold = 0.5):")
display(cm_df)

Retrained Model Evaluation Metrics:


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.66068,1.0,0.66068,0.795674,0.640611,0.501681



Retrained Model Confusion Matrix (Threshold = 0.5):


,expected_label,_0,_1
0,0,0,161284
1,1,0,312516



Baseline Model Evaluation Metrics:


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.66068,1.0,0.66068,0.795674,0.640611,0.501681



Baseline Model Confusion Matrix (Threshold = 0.5):


,expected_label,_0,_1
0,0,0,161284
1,1,0,312516


## Refine features and model

### Subtask:
Refine the features or explore different modeling techniques if necessary, based on the evaluation results.


**Reasoning**:
Based on the lack of improvement with the new features and the limitations of a simple logistic regression model for this problem, I will explore a different model type available in BigQuery ML, specifically a boosted tree classifier, which is often more effective for complex classification tasks.



In [37]:
sql = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET}.churn_boosted_tree_lite`
OPTIONS (
  model_type = 'BOOSTED_TREE_CLASSIFIER',
  input_label_cols = ['churn_next_month'],
  data_split_method = 'SEQ',
  data_split_col = 'month'
) AS
SELECT
  churn_next_month,
  r3_sess,
  r3_min,
  unique_days_watched,
  avg_watch_duration,
  days_since_last_month_start,
  subscription_plan,
  country,
  age,
  month
FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite`
ORDER BY month;
"""
bq.query(sql).result()
print("✓ trained boosted tree model:", f"{PROJECT_ID}.{DATASET}.churn_boosted_tree_lite")

KeyboardInterrupt: 

**Reasoning**:
Now that the boosted tree model is trained, I need to evaluate its performance using ML.EVALUATE, ML.ROC_CURVE, and ML.CONFUSION_MATRIX and store the results for comparison with the previous models.



In [39]:
eval_df_new_model = bq.query(
    f"SELECT * FROM ML.EVALUATE(MODEL `{PROJECT_ID}.{DATASET}.churn_boosted_tree_lite`);"
).result().to_dataframe()

roc_df_new_model = bq.query(
    f"SELECT * FROM ML.ROC_CURVE(MODEL `{PROJECT_ID}.{DATASET}.churn_boosted_tree_lite`);"
).result().to_dataframe()

cm_df_new_model = bq.query(f"""
SELECT *
FROM ML.CONFUSION_MATRIX(
  MODEL `{PROJECT_ID}.{DATASET}.churn_boosted_tree_lite`,
  (SELECT
     churn_next_month,
     r3_sess, r3_min, unique_days_watched, avg_watch_duration, days_since_last_month_start,
     subscription_plan, country, age
   FROM `{PROJECT_ID}.{DATASET}.feat_churn_lite`),
  STRUCT(0.5 AS threshold)
);
""").result().to_dataframe()

print("New Boosted Tree Model Evaluation Metrics:")
display(eval_df_new_model)

print("\nNew Boosted Tree Model Confusion Matrix (Threshold = 0.5):")
display(cm_df_new_model)

print("\nRetrained Logistic Regression Model Evaluation Metrics:")
display(eval_df_retrained)

print("\nRetrained Logistic Regression Model Confusion Matrix (Threshold = 0.5):")
display(cm_df_retrained)

print("\nBaseline Logistic Regression Model Evaluation Metrics:")
display(eval_df)

print("\nBaseline Logistic Regression Model Confusion Matrix (Threshold = 0.5):")
display(cm_df)

New Boosted Tree Model Evaluation Metrics:


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.660666,0.999824,0.660602,0.795608,0.647565,0.485658



New Boosted Tree Model Confusion Matrix (Threshold = 0.5):


,expected_label,_0,_1
0,0,140,161144
1,1,30,312486



Retrained Logistic Regression Model Evaluation Metrics:


...

,expected_label,_0,_1
0,0,0,161284
1,1,0,312516



Baseline Logistic Regression Model Evaluation Metrics:


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.66068,1.0,0.66068,0.795674,0.640611,0.501681



Baseline Logistic Regression Model Confusion Matrix (Threshold = 0.5):


,expected_label,_0,_1
0,0,0,161284
1,1,0,312516


## Summary:

### Data Analysis Key Findings

*   The initial logistic regression model trained with existing features (`r3_sess`, `r3_min`, `subscription_plan`, `country`, `age`) showed poor performance, with an AUC of approximately 0.50 and a high log loss of 0.64. The model predicted the majority class (not churn) for all instances at a 0.5 threshold.
*   New features were engineered from the `watch_history` and `users` tables, including `unique_days_watched`, `avg_watch_duration`, and `days_since_last_month_start`.
*   Retraining the logistic regression model with these new features did not improve its performance. The evaluation metrics and confusion matrix remained identical to the baseline model, still predicting the majority class for all instances at a 0.5 threshold.
*   Exploring a different model type, a boosted tree classifier was trained with the same feature set. This model also did not show significant improvement in overall performance metrics (AUC approximately 0.49, log loss approximately 0.65) compared to the logistic regression models.
*   At a 0.5 threshold, the boosted tree model did predict a small number of churn instances correctly (30 True Positives) but also had a very high number of false positives (161144), indicating a strong bias towards predicting the majority class despite some ability to identify minority class instances.

### Insights or Next Steps

*   The current set of features, even with the additions from `watch_history`, may not be sufficiently discriminative to capture the complex patterns leading to churn. Further feature engineering focusing on more sophisticated behavioral metrics (e.g., content diversity, engagement recency weighted by time, specific event triggers) is needed.
*   Investigating other BigQuery ML models or exploring hyperparameter tuning for the boosted tree model could potentially yield better results, although the feature set appears to be the primary limitation based on current performance.
